In [28]:
import pandas as pd
import re
import ast
from statistics import mean 
import numpy as np
from sklearn.metrics import classification_report, cohen_kappa_score

In [2]:
def conv(s):
    s = s[1:-1]
    dic = {}

    positions=[match.end() for match in re.finditer(r'\b(?:positive|negative|neutral)\b,', s, flags=re.IGNORECASE)]
    for ss in [s[i:j] for i, j in zip([0] + positions, positions + [None])]:
        tmp = ss.split(':')
        try:
            dic[tmp[0]] = tmp[1].replace(",", "")
        except:
            continue

    return dic

def prepros(df, feature):
    df[feature] = df[feature].str.replace('[\n"\']', '', regex=True)
    df[feature] = df[feature].str.replace(r'{\s+', '{', regex=True)
    df[feature] = df[feature].str.replace(r'\s+}', '}', regex=True)
    df[feature] = df[feature].str.replace(r':\s+', ':', regex=True)
    df[feature] = df[feature].str.replace(r',\s+', ',', regex=True)
    df[feature] = df[feature].apply(conv)

    return df

In [21]:
df = pd.read_excel('Human_Annotation_Data.xlsx')

#df.dropna(inplace=True)

for feature in ['sentiments', 'HSent1', 'HSent2']:
    df = prepros(df, feature)

df.head()

,title,text,sentiments,claim_norm_list,who_what_list,HSent1,RelClaim1,RelWhat1,RelWhy1,HSent2,RelClaim2,RelWhat2,RelWhy2
0,"Bloggers stated on January 20, 2020 in a Faceb...","For some, the impeachment trial outcome is a f...","{'nancy pelosi': 'negative', 'donald trump': '...","[""the headline claims u.s. house speaker nancy...","{'what': [""the headline claims u.s. house spea...","{'nancy pelosi': 'neutral', 'donald trump': 'n...","[2,1,2]","[2,2,2]","[1,2]","{'nancy pelosi': 'neutral', 'donald trump': 'n...","[2,0,1]","[2,2,1]","[0,1]"
1,"Ted Cruz stated on December 15, 2020 in a twee...","U.S. Sen. Ted Cruz, R-Texas, took to Twitter T...","{'ted cruz': 'negative', 'donald trump': 'posi...",['the emerging vaccines each were placed throu...,"{'what': [""the guidance issued by the cdc that...","{'ted cruz': 'negative', 'donald trump': 'posi...","[2,2]","[2,2]","[2,2]",{'ted cruz': 'negative'},"[2,2]","[2,2]","[0,0]"
2,"Rebecca Dallet stated on March 2, 2018 in a de...",With the Florida school shooting still fresh o...,"{'ron johnson': 'negative', 'tammy baldwin': '...","[""dallet says that during the supreme court ca...",{'what': ['screnock did win the nra’s endorsem...,"{'ron johnson': 'negative', 'tammy baldwin': '...","[2,2]",[1],[2],"{'ron johnson': 'negative', 'tammy baldwin': '...","[2,2]",[1],[2]
3,"Viral image stated on November 8, 2022 in an I...",Republican U.S. Senate candidate Dr. Mehmet Oz...,{'republicans': 'negative'},['claims that a pennsylvania judge said ballot...,{'what': ['claims that a pennsylvania judge sa...,{'republicans': 'negative'},"[2,2,2]",[2],"[2,2]",{'republicans': 'negative'},"[2,0,2]",[2],"[2,2]"
4,"Viral image stated on October 19, 2022 in an I...",Ukrainian President Volodymyr Zelenskyy is smi...,{'vladimir putin': 'negative'},"[""since russia invaded ukraine, russian presid...","{'what': [""the photo was altered to show ukrai...",{'vladimir putin': 'negative'},"[2,2]",[1],[1],{'vladimir putin': 'negative'},"[2,2]",[1],[0]


In [31]:
scores = {}#{'RelClaim': [], 'RelWhat': [], 'RelWhy': []}

tmp = df.dropna().reset_index(drop=True)
sc = ['RelClaim1', 'RelWhat1', 'RelWhy1', 'RelClaim2', 'RelWhat2', 'RelWhy2']
for feature in sc:
    tmp[feature] = tmp[feature].apply(lambda x: ast.literal_eval(x))

for i in sc:
    for j in tmp[i]:
        try:
            scores[i] += j
        except:
            scores[i] = j

print("Score for Relevancy !!")
for i in scores:
    print(i, " -- ", mean(scores[i]))

print("\n\niAA :P")
for i in range(3):
    print(sc[i], sc[i+3], cohen_kappa_score(scores[sc[i]], scores[sc[i+3]]))

Score for Relevancy !!
RelClaim1  --  1.8780487804878048
RelWhat1  --  1.75
RelWhy1  --  1.6666666666666667
RelClaim2  --  1.6097560975609757
RelWhat2  --  1.6666666666666667
RelWhy2  --  1.3333333333333333


iAA :P
RelClaim1 RelClaim2 0.36379310344827587
RelWhat1 RelWhat2 0.7948717948717949
RelWhy1 RelWhy2 0.475


In [25]:
# Score for entity's recall !!
def recall_calc(Lhuman, Lmodel):
    if(len(Lhuman)==0):
        print("empty :(")
        return 1
    
    return len(Lhuman.intersection(Lmodel)) / len(Lhuman)

recall1 = []
recall2 = []
for i in range(len(df)):
    row = df.iloc[i]
    S_llm = row.sentiments.keys()
    S1 = set(row.HSent1.keys())
    S2 = set(row.HSent2.keys())

    recall1.append(recall_calc(S1, S_llm))
    recall2.append(recall_calc(S2, S_llm))

mean(recall1), mean(recall2)

empty :(
empty :(


(0.9333333333333333, 0.875)

In [26]:
g1 = []
g2 = []
llm = []
for i in range(len(df)):
    row = df.iloc[i]
    for j in row.sentiments:
        try:
            g1.append(row.HSent1[j])
            g2.append(row.HSent2[j])
            llm.append(row.sentiments[j])
        except:
            if(len(g1)==len(g2) and len(g2)==len(llm)):
                print("No issue")
            elif(len(g1)>len(g2)):
                g1 = g1[:-1]
            elif(len(g2)>len(llm)):
                g1 = g1[:-1]
                g2 = g2[:-1]

print(classification_report(g1, llm, digits=5))
print(classification_report(g2, llm, digits=5))

No issue
              precision    recall  f1-score   support

    negative    0.70588   1.00000   0.82759        12
     neutral    1.00000   0.60000   0.75000        10
    positive    1.00000   0.75000   0.85714         4

    accuracy                        0.80769        26
   macro avg    0.90196   0.78333   0.81158        26
weighted avg    0.86425   0.80769   0.80229        26

              precision    recall  f1-score   support

    negative    0.70588   1.00000   0.82759        12
     neutral    1.00000   0.60000   0.75000        10
    positive    1.00000   0.75000   0.85714         4

    accuracy                        0.80769        26
   macro avg    0.90196   0.78333   0.81158        26
weighted avg    0.86425   0.80769   0.80229        26

